In [1]:
#parsing
import os
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
import hashlib
import json
import random

# Data manipulation
import pandas as pd
import numpy as np

#selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#Modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
from sklearn.model_selection import GridSearchCV
import optuna

#Heatmap and VIF
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
change_log_dir = "../changelogs"

In [5]:
# Set up the WebDriver (Selenium)
# Path to your WebDriver (e.g., chromedriver)
driver_path = r'Z:\Python\chromedriver-win64\chromedriver.exe'

# Create a Service object with the path to your chromedriver
service = Service(driver_path)

# Initialize the WebDriver with the Service object
driver = webdriver.Chrome(service=service)

# Open the webpage
url = "https://www.elferspot.com/en/search/?series%5B%5D=911-f-model&series%5B%5D=912&series%5B%5D=911-g-model&series%5B%5D=930&series%5B%5D=964&series%5B%5D=993&series%5B%5D=996&series%5B%5D=997&series%5B%5D=991&series%5B%5D=992&series%5B%5D=911-backdate-modified&series%5B%5D=981&series%5B%5D=982&series%5B%5D=718"
driver.get(url)

# Wait for the page to load (you may need to adjust this for your page's behavior)
time.sleep(1.5)

# Step 1: Handle the cookie popup
try:
    cookie_accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.brlbs-btn-accept-all'))  # Adjust selector
    )
    cookie_accept_button.click()
    print("Cookie popup accepted.")
except:
    print("No cookie popup found or it was not visible in time.")

# Step 2: Scroll the page one full screen height
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)  # Wait for content to load

# Step 3: Handle the second popup (if it appears)
try:
    dont_show_again_link = WebDriverWait(driver, 8).until(
        EC.element_to_be_clickable((By.ID, 'popover_close'))  # Target the "Don't show again" link
    )
    dont_show_again_link.click()
    print("Search agent popup closed.")
except:
    print("No 'Save this search' popup found or it was not visible in time.")

# Step 4: Continue scrolling and waiting for results to load (full-page scrolls)
for _ in tqdm(range(100), desc="Scrolling and loading more listings"):
    # Scroll by the height of the visible viewport (one full screen)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1.5)  # Wait for content to load

# Step 5: Extract all the listing URLs
listing_urls = []
try:
    # Find all elements that contain the listing links
    car_listings = driver.find_elements(By.CSS_SELECTOR, 'a.content-teaser')  # Adjust the selector
    for listing in car_listings:
        url = listing.get_attribute("href")
        if url:
            listing_urls.append(url)

    # Print out the URLs
    print(f"Found {len(listing_urls)} listing URLs:")

    prev_path = "../data/all_listing_urls.csv"
    if os.path.exists(prev_path):
        prev_urls = set(pd.read_csv(prev_path)["Listing_URL"].tolist())
        new_urls = set(listing_urls) - prev_urls
        print(f"New URLs not in previous file: {len(new_urls)}")
    else:
        print("No previous URL file found. All URLs are new.")

except Exception as e:
    print(f"Error extracting listing URLs: {str(e)}")

# Step 6: Create a pandas DataFrame and save to CSV
df = pd.DataFrame(listing_urls, columns=['Listing_URL'])
df.to_csv("../data/all_listing_urls.csv", index=False)
print("Listing URLs have been saved to 'all_listing_urls.csv'.")

# Wait to observe the results or continue interacting
time.sleep(5)

# Close the browser
driver.quit()


Cookie popup accepted.
Search agent popup closed.
Search agent popup closed.


Scrolling and loading more listings: 100%|██████████| 100/100 [02:35<00:00,  1.55s/it]



Found 3520 listing URLs:
New URLs not in previous file: 911
Listing URLs have been saved to 'all_listing_urls.csv'.
New URLs not in previous file: 911
Listing URLs have been saved to 'all_listing_urls.csv'.


In [6]:
# Initialize session for persistent connection reuse
session = requests.Session()

# Headers for the requests
HEADERS = {
    "User-Agent": 
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
}

cookies = {
    "borlabs-cookie": "%7B%22consents%22%3A%7B%22essential%22%3A%5B%22borlabs-cookie%22%2C%22vg-wort%22%2C%22woocommerce%22%2C%22wordfence%22%2C%22wpml%22%5D%2C%22statistics%22%3A%5B%22google-analytics-four%22%2C%22woocommerce-google-analytics%22%5D%2C%22marketing%22%3A%5B%22google-ads%22%2C%22meta-pixel-for-woocommerce%22%2C%22taboola%22%2C%22wc-order-attribution%22%5D%2C%22external-media%22%3A%5B%22pinterest%22%2C%22vimeo%22%2C%22youtube%22%5D%7D%2C%22domainPath%22%3A%22www.elferspot.com%2Fen%2F%22%2C%22expires%22%3A%22Wed%2C%2005%20Nov%202025%2016%3A12%3A58%20GMT%22%2C%22uid%22%3A%22m8drsv73-w32f3qy0-h1xdf0ij-1393u9z0%22%2C%22v3%22%3Atrue%2C%22version%22%3A2%7D",
    "atom-no-inquiry-popover": "true",
    "wp-wpml_current_admin_language_d41d8cd98f00b204e9800998ecf8427e": "en",
    "wp-wpml_current_language": "en"
}


FIELDS_TO_HASH = ["Title", "Price","Mileage","Description","Secondary_Description"] 

def compute_hash(row, fields):
    row_data = {field: row.get(field, "") for field in fields}
    row_json = json.dumps(row_data, sort_keys=True)
    return hashlib.md5(row_json.encode("utf-8")).hexdigest()

def get_field_changes(old_row, new_row, fields):
    changes = []
    for field in fields:
        old = old_row.get(field, "")
        new = new_row.get(field, "")
        if old != new:
            changes.append(f"{field}: '{old}' → '{new}'")
    return "; ".join(changes) if changes else None

def parse_listing(url):
    r = session.get(url, headers=HEADERS, cookies=cookies)
    with open("debug_output.html", "w", encoding="utf-8") as f:
        f.write(r.text)

    if r.status_code != 200:
        return {"URL": url, "Error": f"Failed with status {r.status_code}", "Scraped_At": datetime.today().date()}

    soup = BeautifulSoup(r.text, 'html.parser')
    data = {"URL": url, "Scraped_At": datetime.today().date()}

    title_tag = soup.find("h1")
    if title_tag:
        data["Title"] = title_tag.get_text(strip=True)

    # --- Parse main description from #overview_anchor .col-xs-12.col-md-8 > .content ---
    main_desc = None
    overview_section = soup.find("section", id="overview_anchor")
    if overview_section:
        main_col = overview_section.find("div", class_="col-xs-12 col-md-8")
        if main_col:
            content_div = main_col.find("div", class_="content")
            if content_div:
                main_desc_paragraphs = [
                    p.get_text(strip=True)
                    for p in content_div.find_all("p")
                    if "small" not in (p.get("class") or [])
                ]
                if main_desc_paragraphs:
                    main_desc = "\n".join(main_desc_paragraphs)
    if main_desc:
        data["Description"] = main_desc

    # --- Parse secondary description from .maincontent .container .row .col-xs-12 > .content (not .col-md-8) ---
    secondary_desc = None
    maincontent_section = soup.find("section", class_="maincontent")
    if maincontent_section:
        for row in maincontent_section.find_all("div", class_="row"):
            for col in row.find_all("div", class_="col-xs-12"):
                # Exclude .col-md-8 columns (already parsed above)
                if "col-md-8" in (col.get("class") or []):
                    continue
                content_div = col.find("div", class_="content")
                if content_div:
                    sec_desc_parts = []
                    for elem in content_div.find_all(["p", "ul"], recursive=False):
                        if elem.name == "p":
                            if "small" not in (elem.get("class") or []):
                                sec_desc_parts.append(elem.get_text(strip=True))
                        elif elem.name == "ul":
                            items = [li.get_text(strip=True) for li in elem.find_all("li")]
                            if items:
                                sec_desc_parts.append("\n".join(items))
                    if sec_desc_parts:
                        secondary_desc = "\n".join(sec_desc_parts)
                        break
            if secondary_desc:
                break
    if secondary_desc:
        data["Secondary_Description"] = secondary_desc

    spec_tables = soup.select("table.fahrzeugdaten")
    for table in spec_tables:
        for row in table.select("tr"):
            label = row.select_one("td.label")
            content = row.select_one("td.content")
            if label and content:
                key = label.get_text(strip=True).rstrip(":")
                value = content.get_text(strip=True)
                data[key] = value

    price_tag = soup.select_one("div.sidebar-section .price span.p")
    if price_tag:
        data["Price"] = price_tag.get_text(strip=True)

    return data

def parse_all_listings(listing_urls, max_workers=6):
    listings_data = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(parse_listing, url): url for url in listing_urls}
        for future in tqdm(as_completed(future_to_url), total=len(listing_urls), desc="Parsing listings"):
            listing_data = future.result()
            listings_data.append(listing_data)
    return pd.DataFrame(listings_data)


In [7]:
# --- Parse new listings: Only URLs not in all_listings_bronze.xlsx ---
excel_path = "../data/all_listings_bronze.xlsx"
df_urls = pd.read_csv("../data/all_listing_urls.csv")
all_urls = set(df_urls['Listing_URL'].tolist())

# Load existing data
if os.path.exists(excel_path):
    df_existing = pd.read_excel(excel_path)
    df_existing.set_index("URL", inplace=True)
else:
    df_existing = pd.DataFrame().set_index("URL")

existing_urls = set(df_existing.index)
new_urls = list(all_urls - existing_urls)

if new_urls:
    print(f"🔍 Found {len(new_urls)} new listings (not in all_listings_bronze.xlsx). Parsing...")
    df_new = parse_all_listings(new_urls, max_workers=8)
    df_new['Data_Hash'] = df_new.apply(lambda row: compute_hash(row, FIELDS_TO_HASH), axis=1)
    df_new.set_index("URL", inplace=True)
    df_existing = pd.concat([df_existing, df_new])
    print(f"✅ Appended {len(new_urls)} new listings.")

🔍 Found 885 new listings (not in all_listings_bronze.xlsx). Parsing...


Parsing listings: 100%|██████████| 885/885 [01:30<00:00,  9.81it/s]


✅ Appended 885 new listings.


In [ ]:
# --- Update existing listings ---
update_urls = list(existing_urls & all_urls)
change_log = []
if update_urls:
    print(f"🔄 Checking {len(update_urls)} existing listings for updates...")
    df_updates = parse_all_listings(update_urls)
    df_updates.set_index("URL", inplace=True)
    for url in update_urls:
        if url in df_existing.index:
            old_row = df_existing.loc[url].to_dict()
            new_row = df_updates.loc[url].to_dict()
            # --- Protection: skip update if error present ---
            if "Error" in new_row and new_row["Error"]:
                print(f"⚠️ Skipping update for {url} due to error: {new_row['Error']}")
                continue
            old_hash = old_row.get("Data_Hash", "")
            new_hash = compute_hash(new_row, FIELDS_TO_HASH)
            if old_hash != new_hash:
                changes = get_field_changes(old_row, new_row, FIELDS_TO_HASH)
                # --- Ignore if only Price changed and it is now 'reserved', empty, or NaN ---
                price_changed = old_row.get("Price", "") != new_row.get("Price", "")
                only_price_changed = (
                    price_changed and
                    all(
                        (old_row.get(f, "") == new_row.get(f, "")) 
                        for f in FIELDS_TO_HASH if f != "Price"
                    )
                )
                new_price = str(new_row.get("Price", "")).strip().lower()
                if only_price_changed and (new_price == "reserved" or new_price == "" or pd.isna(new_row.get("Price", ""))):
                    continue  # Skip this update
                # --- Type compatibility fix ---
                for col in df_existing.columns:
                    if col in new_row:
                        try:
                            dtype = df_existing[col].dtype
                            if pd.isna(new_row[col]):
                                continue
                            if "datetime" in str(dtype):
                                new_row[col] = pd.to_datetime(new_row[col], errors="coerce")
                            elif "float" in str(dtype):
                                new_row[col] = float(new_row[col])
                            elif "int" in str(dtype):
                                new_row[col] = int(float(new_row[col]))
                            else:
                                new_row[col] = str(new_row[col])
                        except Exception:
                            pass
                new_row['Data_Hash'] = new_hash
                df_existing.loc[url] = pd.Series(new_row)
                change_log.append({"URL": url, "Changes": changes, "Date": datetime.now().strftime("%Y-%m-%d-%H:%M:%S")})
    if change_log:
        log_file = os.path.join(change_log_dir, f"changelog_{datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.xlsx")
        pd.DataFrame(change_log).to_excel(log_file, index=False)
        print(f"📝 Saved changelog with {len(change_log)} updates to {log_file}")

🔄 Checking 2794 existing listings for updates...


Parsing listings: 100%|██████████| 2794/2794 [15:45<00:00,  2.96it/s]


📝 Saved changelog with 894 updates to ../changelogs\changelog_2025-09-12-19-35-25.xlsx
✅ Updated listing data saved to '../data/all_listings_bronze.xlsx'. Total listings: 5772


In [8]:
# Save the updated listings
df_existing.reset_index(inplace=True)
df_existing.to_excel(excel_path, index=False)
print(f"✅ Updated listing data saved to '{excel_path}'. Total listings: {len(df_existing)}")

✅ Updated listing data saved to '../data/all_listings_bronze.xlsx'. Total listings: 6657


In [ ]:
def update_descriptions_parallel(df, max_workers=8):
    urls = df["URL"].tolist()
    results = {}

    def fetch_desc(url):
        try:
            parsed = parse_listing(url)
            desc = parsed.get("Description", None)
            sec_desc = parsed.get("Secondary_Description", None)
            return url, desc, sec_desc
        except Exception as e:
            print(f"Failed to update {url}: {e}")
            return url, None, None

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_desc, url): url for url in urls}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Updating descriptions"):
            url, desc, sec_desc = future.result()
            idx = df.index[df["URL"] == url]
            if len(idx):
                if desc is not None:
                    df.at[idx[0], "Description"] = desc
                if sec_desc is not None:
                    df.at[idx[0], "Secondary_Description"] = sec_desc

    return df

df_existing = update_descriptions_parallel(df_existing)
df_existing.to_excel(excel_path, index=False)
print(f"✅ Descriptions reparsed and bronze layer updated at '{excel_path}'.")

Updating descriptions: 100%|██████████| 4290/4290 [07:54<00:00,  9.04it/s]



✅ Descriptions reparsed and bronze layer updated at '../data/all_listings_bronze.xlsx'.
